In [1]:
print("OK")

OK


In [2]:
import boto3

In [3]:
s3 = boto3.client("s3")

In [4]:
response = s3.list_objects_v2(Bucket='llm-fine-tune-dataset-mrv',Prefix='dataset/')

In [5]:
for obj in response.get('Contents',[]):
    print(obj)

{'Key': 'dataset/', 'LastModified': datetime.datetime(2025, 11, 11, 15, 48, 52, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'dataset/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2025, 11, 11, 19, 43, 52, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [6]:
for obj in response.get('Contents',[]):
    print(obj['Key'])

dataset/
dataset/pharma_instruction_data.csv


In [7]:
!which python

/home/ec2-user/anaconda3/envs/python3/bin/python


In [8]:
!source /home/ec2-user/anaconda3/bin/activate python3

In [9]:
dataset_path = 's3://llm-fine-tune-dataset-mrv/dataset/pharma_instruction_data.csv'

In [10]:
from datasets import load_dataset

In [11]:
dataset_loaded =load_dataset("csv",data_files={"train":dataset_path})

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:286: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
dataset_loaded

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 5
    })
})

In [13]:
print(dataset_loaded["train"])

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})


In [14]:
print(dataset_loaded["train"][0])

{'instruction': 'Explain the mechanism of action of Metformin.', 'input': None, 'output': 'Metformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'}


In [15]:
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    return {"text": prompt}

In [17]:
dataset = dataset_loaded.map(format_example)

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 5
    })
})

In [20]:
dataset['text'][0]

KeyError: 'text'